In [5]:
rm(list=ls())
# library(lightgbm)
library(data.table)
library(tictoc)
set.seed(0)

mae <- function( a,b)return(mean(abs(a-b)))

## Mean of Log MAE across type
mlmae <- function(actuals, preds, types){
  sum_lmae <- 0
  n_types <- length(unique(types))
  
  # calculating MAE for each type
  for (type in unique(types)){
    sc2 <- mae(actuals[types == type], preds[types == type])
    sc <- log(sc2)
    print( paste( type, sc, sc2 )  )
    sum_lmae <- sum_lmae + sc
  }
  # Mean MAE
  return (sum_lmae / n_types)
}

In [7]:
train <- fread("../../data/input/train.csv")
test  <- fread("../../data/input/test.csv" )
sub <- fread('../../data/input/sample_submission.csv')

#Use ID as groupKfold = 5 folds
train[, ID := as.integer(.GRP)%%4 , by="molecule_name" ]
test[ , ID := as.integer(.GRP)%%4 , by="molecule_name" ]
dim(train)
dim(test)
head(train,30)

[1] 4658147       7

[1] 2505542       6

id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,ID
<int>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>
0,dsgdb9nsd_000001,1,0,1JHC,84.80760,1
1,dsgdb9nsd_000001,1,2,2JHH,-11.25700,1
2,dsgdb9nsd_000001,1,3,2JHH,-11.25480,1
3,dsgdb9nsd_000001,1,4,2JHH,-11.25430,1
4,dsgdb9nsd_000001,2,0,1JHC,84.80740,1
5,dsgdb9nsd_000001,2,3,2JHH,-11.25410,1
6,dsgdb9nsd_000001,2,4,2JHH,-11.25480,1
7,dsgdb9nsd_000001,3,0,1JHC,84.80930,1
8,dsgdb9nsd_000001,3,4,2JHH,-11.25430,1


In [9]:
structures_0 <- fread("../../data/input/structures.csv")
structures_1 <- copy(structures_0)

setnames(structures_0, names(structures_0)[-1], paste0(names(structures_0)[-1], "_0"))
setnames(structures_1, names(structures_1)[-1], paste0(names(structures_1)[-1], "_1"))
setnames(structures_0, names(structures_0)[-c(1:2)], paste0("structure_", names(structures_0)[-c(1:2)]))
setnames(structures_1, names(structures_1)[-c(1:2)], paste0("structure_", names(structures_1)[-c(1:2)]))

train <- merge(train, structures_0, all.x = T,
               by = c("molecule_name", "atom_index_0"))
train <- merge(train, structures_1, all.x = T,
               by = c("molecule_name", "atom_index_1"))
test  <- merge(test , structures_0, all.x = T,
               by = c("molecule_name", "atom_index_0"))
test  <- merge(test , structures_1, all.x = T,
               by = c("molecule_name", "atom_index_1"))

train_test <- rbind(train, test, use.names = T, fill = T)

rm(train, test); gc()
dim(train_test)
head(train_test)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,727993,38.9,1168576,62.5,1168576,62.5
Vcells,131286777,1001.7,256876995,1959.9,225523751,1720.7


[1] 7163689      15

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,structure_z_0,structure_atom_1,structure_x_1,structure_y_1,structure_z_1
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,0.0019761204,C,-0.01269814,1.085804,0.0080009958
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,0.0002765748,C,-0.01269814,1.085804,0.0080009958
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,-0.8766437152,C,-0.01269814,1.085804,0.0080009958
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,0.9063972942,C,-0.01269814,1.085804,0.0080009958
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,0.0019761204,H,1.01173084,1.463751,0.0002765748
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,0.0019761204,H,-0.54081507,1.447527,-0.8766437152


In [10]:
train_test[,typei:=.GRP, by="type"]

train_test[, N1 := length(unique(atom_index_1)) ,
           by=c("molecule_name","atom_index_0")  ]
train_test[, N2 := length(unique(atom_index_0)) ,
           by=c("molecule_name","atom_index_1")  ]

dt0 <- train_test[, paste(type,collapse=" "),
                  by=c("molecule_name","atom_index_0") ]
dt1 <- train_test[, paste(type,collapse=" "),
                  by=c("molecule_name","atom_index_1") ]
setnames(dt0,colnames(dt0), c("molecule_name","atom_index","V1") )
setnames(dt1,colnames(dt1), c("molecule_name","atom_index","V1") )

dt <- rbind( dt0, dt1  )
dt[, link := .GRP , by="V1" ]
dt[, N := .N, by="link"]
dt <- dt[, list(group=link[1]) , keyby="V1" ]
dt0[, link := dt[J(dt0$V1)]$group ]
dt1[, link := dt[J(dt1$V1)]$group ]
setkeyv(dt0, c("molecule_name","atom_index"))
setkeyv(dt1, c("molecule_name","atom_index"))

train_test[, link0 := dt0[J(train_test$molecule_name, train_test$atom_index_0)]$link ]
train_test[, link1 := dt1[J(train_test$molecule_name, train_test$atom_index_1)]$link ]
train_test[, linkN := .N, by=c("link0","link1") ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,structure_atom_1,structure_x_1,structure_y_1,structure_z_1,typei,N1,N2,link0,link1,linkN
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,C,-0.01269814,1.085804,0.0080009958,1,4,4,1,9878,1
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,C,-0.01269814,1.085804,0.0080009958,1,3,4,2,9878,1
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,C,-0.01269814,1.085804,0.0080009958,1,2,4,3,9878,1
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,C,-0.01269814,1.085804,0.0080009958,1,1,4,4,9878,1
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,H,1.01173084,1.463751,0.0002765748,2,4,1,1,8,1
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,H,-0.54081507,1.447527,-0.8766437152,2,4,2,1,9879,1


In [11]:
# difference and distance features
train_test[, ":="(dist_xyz = sqrt((structure_x_0 - structure_x_1) ^ 2 +
                                    (structure_y_0 - structure_y_1) ^ 2 +
                                    (structure_z_0 - structure_z_1) ^ 2)
)]

train_test[, inv_dist0 := 1/sum(1/dist_xyz^3) ,
           by=c("molecule_name","atom_index_0") ]
train_test[, inv_dist1 := 1/sum(1/dist_xyz^3) ,
           by=c("molecule_name","atom_index_1") ]
train_test[, inv_distP := (inv_dist0*inv_dist1)/(inv_dist0+inv_dist1) ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,typei,N1,N2,link0,link1,linkN,dist_xyz,inv_dist0,inv_dist1,inv_distP
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,1,4,4,1,9878,1,1.091953,0.7709034,0.3254976,0.2288645
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,1,3,4,2,9878,1,1.091952,0.8922213,0.3254976,0.2384918
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,1,2,4,3,9878,1,1.091946,1.0588319,0.3254976,0.2489633
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,1,1,4,4,9878,1,1.091948,1.3019830,0.3254976,0.2603978
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,2,4,1,1,8,1,1.783120,0.7709034,5.6694579,0.6786272
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,2,4,2,1,9879,1,1.783147,0.7709034,2.8348855,0.6060873


In [12]:
atomic_radius <- data.table(
  atom = c('H','C','N','O','F'),
  R = c(0.38,0.77,0.75,0.73,0.71),
  E = c(2.2,2.55,3.04,3.44,3.98)
  ,key="atom"
)
train_test[, R0 := atomic_radius[J(train_test$structure_atom_0)]$R  ]
train_test[, R1 := atomic_radius[J(train_test$structure_atom_1)]$R  ]
train_test[, E0 := atomic_radius[J(train_test$structure_atom_0)]$E  ]
train_test[, E1 := atomic_radius[J(train_test$structure_atom_1)]$E  ]

train_test[, inv_dist0R := 1/sum(1/((dist_xyz-R0-R1)^2)) ,
           by=c("molecule_name","atom_index_0") ]
train_test[, inv_dist1R := 1/sum(1/((dist_xyz-R0-R1)^2)) ,
           by=c("molecule_name","atom_index_1") ]
train_test[, inv_distPR := (inv_dist0R*inv_dist1R)/(inv_dist0R+inv_dist1R) ]

train_test[, inv_dist0E := 1/sum(1/((dist_xyz*(0.5*E0+0.5*E1))^2)) ,
           by=c("molecule_name","atom_index_0") ]
train_test[, inv_dist1E := 1/sum(1/((dist_xyz*(0.5*E0+0.5*E1))^2)) ,
           by=c("molecule_name","atom_index_1") ]
train_test[, inv_distPE := (inv_dist0E*inv_dist1E)/(inv_dist0E+inv_dist1E) ]

train_test[, linkM0:= mean(inv_dist0), by=c("link0") ]
train_test[, linkM1:= mean(inv_dist0), by=c("link1") ]
train_test[, linkM0:= linkM0 - inv_dist0 ]
train_test[, linkM1:= linkM1 - inv_dist1 ]

head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,E0,E1,inv_dist0R,inv_dist1R,inv_distPR,inv_dist0E,inv_dist1E,inv_distPE,linkM0,linkM1
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,2.2,2.55,0.003337222,0.0008424608,0.0006726536,2.910143,1.681405,1.065682,0,0.6804872
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,2.2,2.55,0.003348061,0.0008424608,0.0006730928,3.588810,1.681405,1.144971,0,0.6804872
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,2.2,2.55,0.003359407,0.0008424608,0.0006735502,4.680201,1.681405,1.237001,0,0.6804872
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,2.2,2.55,0.003370088,0.0008424608,0.0006739784,6.725596,1.681405,1.345123,0,0.6804872
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,2.2,2.20,0.003337222,1.0467740352,0.0033266166,2.910143,15.388858,2.447334,0,-4.8862267
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,2.2,2.20,0.003337222,0.5234205990,0.0033160796,2.910143,7.694712,2.111553,0,-2.0069312


In [13]:
train_test <- merge(train_test, train_test[, 
                                  .(
                                   min_molecule_atom_0_dist_xyz  = min(dist_xyz),
                                   mean_molecule_atom_0_dist_xyz = mean(dist_xyz),
                                   max_molecule_atom_0_dist_xyz  = max(dist_xyz),
                                   sd_molecule_atom_0_dist_xyz   = sd(dist_xyz)
), .(molecule_name, atom_index_0)], by = c("molecule_name", "atom_index_0"))

train_test <- merge(train_test, train_test[, 
                                  .(
                                   min_molecule_atom_1_dist_xyz  = min(dist_xyz),
                                   mean_molecule_atom_1_dist_xyz = mean(dist_xyz),
                                   max_molecule_atom_1_dist_xyz  = max(dist_xyz),
                                   sd_molecule_atom_1_dist_xyz   = sd(dist_xyz)
), .(molecule_name, atom_index_1)], by = c("molecule_name", "atom_index_1"))

In [15]:
dstruct = fread('../../data/input/structures.csv')
dstruct[,i:=.I]
list_atoms = dstruct[,.(molecule_name,atom_index,i)]
md = as.matrix(dstruct[,.(x,y,z)])
atom_type =unlist(dstruct[,atom])
rm(dstruct)
system.time(
    {
     aij<-merge(list_atoms,list_atoms,by='molecule_name',allow.cartesian=TRUE)
     aij[,atom_index.y:=NULL]
     colnames(aij)<-c('molecule_name','atom_index','i','j')
     aij<-aij[i!=j]
     aij[,atj:=atom_type[j]]
    }
)
rs = md[aij[,i],]-md[aij[,j],]
system.time(aij[,d:=sqrt(rowSums(rs*rs))])
system.time(daij<-aij[,.(coulomb=sum(1/d),vander=sum(1/d/d/d),yukawa=sum(exp(-d))),by=.(molecule_name,atom_index,atj)])
system.time(daij2<-dcast(daij,molecule_name+atom_index~atj,value.var=c('coulomb','yukawa','vander')))
daij2[is.na(daij2)]=0

rm(list_atoms,atom_type,aij,rs,daij)

dataCY=merge(train_test[,.(molecule_name,atom_index_0,atom_index_1)],daij2,by.x=c('molecule_name','atom_index_0'),by.y=c('molecule_name','atom_index'),sort=F)
dataCY=merge(dataCY,daij2,by.x=c('molecule_name','atom_index_1'),by.y=c('molecule_name','atom_index'),sort=F)

rm(daij2)

train_test=cbind(train_test,dataCY[,-3,with=F])

   user  system elapsed 
  3.438   0.639   1.487 

   user  system elapsed 
  0.452   0.352   0.805 

   user  system elapsed 
 15.315   0.137  12.559 

   user  system elapsed 
  3.906   0.085   1.001 

In [16]:
train_test[train_test$structure_atom_1=='C',
           countC0:= .N,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='H',
           countH0:= .N,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='N',
           countN0:= .N,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='C',
           countC1 := .N,
           by=c("molecule_name","atom_index_1")  ]
train_test[train_test$structure_atom_1=='H',
           countH1 := .N,
           by=c("molecule_name","atom_index_1")  ]
train_test[train_test$structure_atom_1=='N',
           countN1 := .N,
           by=c("molecule_name","atom_index_1")  ]

train_test[train_test$structure_atom_1=='C',
           distC0 := sum(dist_xyz)/countC0  ,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='H',
           distH0 := sum(dist_xyz)/countH0  ,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='N',
           distN0 := sum(dist_xyz)/countN0  ,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='C',
           distC1 := sum(dist_xyz)/countC1  ,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='H',
           distH1 := sum(dist_xyz)/countH1  ,
           by=c("molecule_name","atom_index_0")  ]
train_test[train_test$structure_atom_1=='N',
           distN1 := sum(dist_xyz)/countN1  ,
           by=c("molecule_name","atom_index_0")  ]

train_test[, countC0:= NULL  ]
train_test[, countH0:= NULL  ]
train_test[, countN0:= NULL  ]
train_test[, countC1:= NULL  ]
train_test[, countH1:= NULL  ]
train_test[, countN1:= NULL  ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,vander_F.y,vander_H.y,vander_N.y,vander_O.y,distC0,distH0,distN0,distC1,distH1,distN1
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,0,3.0722189,0,0,1.091953,NA,NA,0.2729883,NA,NA
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,0,3.0722189,0,0,1.091952,NA,NA,0.2729879,NA,NA
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,0,3.0722189,0,0,1.091946,NA,NA,0.2729866,NA,NA
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,0,3.0722189,0,0,1.091948,NA,NA,0.2729869,NA,NA
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,0,0.5291314,0,0,NA,1.783141,NA,NA,5.349424,NA
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,0,0.5291233,0,0,NA,1.783141,NA,NA,2.674712,NA


In [17]:
train_test[, pos := 1:.N, by=c("molecule_name","atom_index_0") ]

sort_dist <- function( par, typ, k="H" ){
  res <- matrix( 999, nrow=length(par), ncol=13 )
  par[ typ!=k ] <- 999
  for( i in 1:nrow(res) ){
    res[i, 1:length(par) ] <- par
    res[i, i ] <- 999
    res[i,] <- res[i, order(res[i,],decreasing=F) ]
  }
  res[res==999] <- NA
  return( list( res[,1],res[,2],res[,3],res[,4],res[,5] ) )
}

dt <- train_test[, sort_dist(par=dist_xyz,typ=structure_atom_1,k="H") ,
                 keyby=c("molecule_name","atom_index_0") ]
dt[, pos := 1:.N, by=c("molecule_name","atom_index_0") ]
setkeyv( dt, c("molecule_name","atom_index_0","pos")  )

train_test[, adH1 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V1 ]
train_test[, adH2 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V2 ]
train_test[, adH3 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V3 ]
train_test[, adH4 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V4 ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,distH0,distN0,distC1,distH1,distN1,pos,adH1,adH2,adH3,adH4
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,NA,NA,0.2729883,NA,NA,1,1.783120,1.783147,1.783157,NA
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,NA,NA,0.2729879,NA,NA,1,1.783148,1.783158,NA,NA
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,NA,NA,0.2729866,NA,NA,1,1.783148,NA,NA,NA
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,NA,NA,0.2729869,NA,NA,1,NA,NA,NA,NA
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,1.783141,NA,NA,5.349424,NA,2,1.783147,1.783157,NA,NA
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,1.783141,NA,NA,2.674712,NA,3,1.783120,1.783157,NA,NA


In [18]:
dt <- train_test[, sort_dist(par=dist_xyz,typ=structure_atom_1,k="C") ,
                 keyby=c("molecule_name","atom_index_0") ]
dt[, pos := 1:.N, by=c("molecule_name","atom_index_0") ]
setkeyv( dt, c("molecule_name","atom_index_0","pos")  )

train_test[, adC1 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V1 ]
train_test[, adC2 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V2 ]
train_test[, adC3 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V3 ]
train_test[, adC4 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V4 ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,distN1,pos,adH1,adH2,adH3,adH4,adC1,adC2,adC3,adC4
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,NA,1,1.783120,1.783147,1.783157,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,NA,1,1.783148,1.783158,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,NA,1,1.783148,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,NA,1,NA,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,NA,2,1.783147,1.783157,NA,NA,1.091953,NA,NA,NA
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,NA,3,1.783120,1.783157,NA,NA,1.091953,NA,NA,NA


In [19]:
dt <- train_test[, sort_dist(par=dist_xyz,typ=structure_atom_1,k="N") ,
                 keyby=c("molecule_name","atom_index_0") ]
dt[, pos := 1:.N, by=c("molecule_name","atom_index_0") ]
setkeyv( dt, c("molecule_name","atom_index_0","pos")  )

train_test[, adN1 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V1 ]
train_test[, adN2 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V2 ]
train_test[, adN3 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V3 ]
train_test[, adN4 := dt[J(train_test$molecule_name, train_test$atom_index_0, train_test$pos)]$V4 ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,adH3,adH4,adC1,adC2,adC3,adC4,adN1,adN2,adN3,adN4
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,1.783157,NA,NA,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,NA,NA,1.091953,NA,NA,NA,NA,NA,NA,NA
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,NA,NA,1.091953,NA,NA,NA,NA,NA,NA,NA


In [21]:
structure <- fread('../../data/input/structures.csv')
dt <- structure[, .N, keyby= c("molecule_name","atom") ]
dt1 <- dt[ atom=='C' ]
dt2 <- dt[ atom=='H' ]
dt3 <- dt[ atom=='N' ]
dt4 <- dt[ atom=='F' ]
dt5 <- dt[ atom=='O' ]

train_test[, NC := dt1[J(train_test$molecule_name)]$N ]
train_test[, NH := dt2[J(train_test$molecule_name)]$N ]
train_test[, NN := dt3[J(train_test$molecule_name)]$N ]
train_test[, NF := dt4[J(train_test$molecule_name)]$N ]
train_test[, NO := dt5[J(train_test$molecule_name)]$N ]
head(train_test)

molecule_name,atom_index_1,atom_index_0,id,type,scalar_coupling_constant,ID,structure_atom_0,structure_x_0,structure_y_0,⋯,adC4,adN1,adN2,adN3,adN4,NC,NH,NN,NF,NO
<chr>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
dsgdb9nsd_000001,0,1,0,1JHC,84.8076,1,H,0.002150416,-0.006031318,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
dsgdb9nsd_000001,0,2,4,1JHC,84.8074,1,H,1.011730843,1.463751162,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
dsgdb9nsd_000001,0,3,7,1JHC,84.8093,1,H,-0.540815069,1.447526614,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
dsgdb9nsd_000001,0,4,9,1JHC,84.8095,1,H,-0.523813634,1.437932644,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
dsgdb9nsd_000001,2,1,1,2JHH,-11.2570,1,H,0.002150416,-0.006031318,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
dsgdb9nsd_000001,3,1,2,2JHH,-11.2548,1,H,0.002150416,-0.006031318,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA


In [22]:
features <- colnames(train_test)[c(1:ncol(train_test))]
features <- features[ !features %in% c("molecule_name",
                                       "atom_index_0","atom_index_1",
                                       "type","scalar_coupling_constant",
                                       "ID",
                                       "structure_atom_0","structure_atom_1",
                                       "structure_x_0","structure_y_0","structure_z_0",
                                       "structure_x_1","structure_y_1","structure_z_1",
                                       "typei","pos",
                                       "R0","R1","E0","E1"
                                       ) ]
length(features);features
head(train_test[,features,with=F])

[1] 79

[1] "id"                            "N1"                           
 [3] "N2"                            "link0"                        
 [5] "link1"                         "linkN"                        
 [7] "dist_xyz"                      "inv_dist0"                    
 [9] "inv_dist1"                     "inv_distP"                    
[11] "inv_dist0R"                    "inv_dist1R"                   
[13] "inv_distPR"                    "inv_dist0E"                   
[15] "inv_dist1E"                    "inv_distPE"                   
[17] "linkM0"                        "linkM1"                       
[19] "min_molecule_atom_0_dist_xyz"  "mean_molecule_atom_0_dist_xyz"
[21] "max_molecule_atom_0_dist_xyz"  "sd_molecule_atom_0_dist_xyz"  
[23] "min_molecule_atom_1_dist_xyz"  "mean_molecule_atom_1_dist_xyz"
[25] "max_molecule_atom_1_dist_xyz"  "sd_molecule_atom_1_dist_xyz"  
[27] "coulomb_C.x"                   "coulomb_F.x"                  
[29] "coulomb_H.x"                   "coulomb_N.x"                  
[31] "coulomb_O.x"                   "yukawa_C.x"                   
[33] "yukawa_F.x"                    "yukawa_H.x"                   
[35] "yukawa_N.x"                    "yukawa_O.x"                   
[37] "vander_C.x"                    "vander_F.x"                   
[39] "vander_H.x"                    "vander_N.x"                   
[41] "vander_O.x"                    "coulomb_C.y"                  
[43] "coulomb_F.y"                   "coulomb_H.y"                  
[45] "coulomb_N.y"                   "coulomb_O.y"                  
[47] "yukawa_C.y"                    "yukawa_F.y"                   
[49] "yukawa_H.y"                    "yukawa_N.y"                   
[51] "yukawa_O.y"                    "vander_C.y"                   
[53] "vander_F.y"                    "vander_H.y"                   
[55] "vander_N.y"                    "vander_O.y"                   
[57] "distC0"                        "distH0"                       
[59] "distN0"                        "distC1"                       
[61] "distH1"                        "distN1"                       
[63] "adH1"                          "adH2"                         
[65] "adH3"                          "adH4"                         
[67] "adC1"                          "adC2"                         
[69] "adC3"                          "adC4"                         
[71] "adN1"                          "adN2"                         
[73] "adN3"                          "adN4"                         
[75] "NC"                            "NH"                           
[77] "NN"                            "NF"                           
[79] "NO"

id,N1,N2,link0,link1,linkN,dist_xyz,inv_dist0,inv_dist1,inv_distP,⋯,adC4,adN1,adN2,adN3,adN4,NC,NH,NN,NF,NO
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
0,4,4,1,9878,1,1.091953,0.7709034,0.3254976,0.2288645,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
4,3,4,2,9878,1,1.091952,0.8922213,0.3254976,0.2384918,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
7,2,4,3,9878,1,1.091946,1.0588319,0.3254976,0.2489633,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
9,1,4,4,9878,1,1.091948,1.3019830,0.3254976,0.2603978,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
1,4,1,1,8,1,1.783120,0.7709034,5.6694579,0.6786272,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA
2,4,2,1,9879,1,1.783147,0.7709034,2.8348855,0.6060873,⋯,NA,NA,NA,NA,NA,1,4,NA,NA,NA


In [24]:
fwrite( train_test, "../../data/feature/giba-r-data-table-simple-features-1-17-lb.r.csv" )